# CVE Mitre Introduction

Vulnerabilities data are available in three different sources: **CVE Mitre, NVD and CVE Details**, being created and annotated through the data sources in this respective order. Launched in 1999 when most information security tools used their own databases with their own names for security vulnerabilities, the Common Vulnerabilities and Exposures (CVE) by Mitre documents known vulnerabilities manually for public usage. Each vulnerability contains a description, is uniquely identified by a CVE ID, and may also include fields specifying the vulnerable software, version and vendors affected by it. If a set of vulnerabilities are similar, but occur for different software, they can have different CVE-IDs, and contain the same weakness ID (CWE ID). When created by CVE Mitre, each vulnerability may or not be annotated with a weakness ID(CWE ID),but when available they can serve to group similar vulnerabilities conceptually,and observe how they have been ‘instantiated’ in different software, version or vendor. CVE Mitre’s vulnerabilities are then annotated with severity scores, fix information, and impact ratings in the National Vulnerability Database(NVD),and made available for download as XML feeds. CVE Details was created to provide a user-friendly interface to NVD’s XML feeds. For instance, using vulnerabilities’ CWE IDs and keyword matching, it defines 13 vulnerability types to facilitate browsing vulnerabilities. Since CVE Details warns about inconsistencies in NVD XML Feeds (e.g.same vendor’s software having different names), and irrelevant entries to our purposes (i.e. reserved, duplicates and removed entries), we downloaded all software vulnerabilities to date from the three sources to define our vulnerability dataset and ensure consistency.

## Motivation

The CVE Mitre database has information about the reference (or the source) of the vulnerability. There are various sources, and the database provides information about the Url it is reported from the description of the attack (with an ID associated). It is important to identify the right sources of vulnerabilities and this notebook aims to help choose the sources and and filter the chosen ones into a new file. 

# Method

The files provided by the CVE Mitre website are in CVRF(XML)format and can be found http://cve.mitre.org/data/downloads/index.html . The XML schema is built such that it encapsulates tables within a table. We will parse through the tree to reach the required child node and perform pattern matching using regular expressions. This will enable us to extract the right fields and write it onto a file(file1). The other unfiltered sources are writen into another file(file2), from where they can be fetched if felt they are to be considered.

In [181]:
#import Element tree for parsing xml
from xml.etree.ElementTree import ElementTree 
import csv
import re
import glob

#parsing the tree and fetching root node
table_root = "{http://www.icasi.org/CVRF/schema/vuln/1.1}"

The data is very complex and filled with different types of references. On initial evaluation, the data was found to not be completely described by one type of regular expression. The data has over 60 different types of references and each of them vary in its composition. The regex has been formed in a way that it identifies only the IDs of reference types and does not confuse the occurance of that word in other sentences(or descriptions of references). This process of filtering and verification for accuracy is semi automated.

In [232]:
#creating variables for regex search
BID_regex = "BID:(\d+)"
SECTRACK_regex = "SECTRACK:(\d+)"
MS_regex = "MS:[A-Z]*[0-9]*-[0-9]*"
REDHAT_regex = "REDHAT:RHSA-[0-9]*:[0-9]*"
GENTOO_regex = "GENTOO:GLSA-[0-9]*-[0-9]*"
DEBIAN_regex = "DEBIAN:DSA-(\d+)"
SECUNIA_regex = "SECUNIA:(\d+)"
TURBO_regex = "TURBO:TLSA-[0-9]*-[0-9]*"
AIXAPAR_regex = "AIXAPAR:[A-Z]*[0-9]*"
ALLAIRE_regex = "ALLAIRE:[A-Z]*[0-9]*-[0-9]*"
AUSCERT_regex = "AUSCERT:[A-Z]*-[0-9]*.[0-9]*"
BEA_regex = "BEA:BEA[0-9]*-[0-9]*.[0-9]*"
CIAC_regex = "CIAC:[A-Z]*-[0-9]*"
CONECTIVA_regex = "^CONECTIVA"
OSVDB_regex = "OSVDB:(\d+)"
CERT_regex = "^CERT:"
CERT_VN_regex = "^CERT-VN"
APPLE_regex = "^APPLE"
CALDERA_regex = "^CALDERA"
EXPLOIT_regex = "^EXPLOIT"
BUGTRAQ_regex = "^BUGTRAQ"
CISCO_regex = "^CISCO"
FEDORA_regex = "^FEDORA"
FULLDISC_regex = "^FULLDISC"
MISC_regex = "^MISC"
OVAL_regex = "^OVAL"
IBM_regex = "^IBM"
OPENBSD_regex = "^OPENBSD"
VIM_regex = "^VIM"
SUN_regex = "^SUN"
NAI_regex = "^NAI"
CONFIRM_regex = "^CONFIRM"

#creating headers for file
header_file1= ["CVE ID","BID Description","BID Url","SECTRACK Description",
               "SECTRACK Url","MS Description","MS Url","REDHAT Description",
               "REDHAT Url","DEBIAN Description","DEBIAN Url","GENTOO Description",
               "GENTOO Url","SECUNIA Description","SECUNIA Url","TURBO Description",
               "TURBO Url","AIXAPAR Description","AIXAPAR Url","ALLAIRE Description",
               "ALLAIRE Url","APPLE Description","APPLE Url","ATSTAKE Description",
               "ATSTAKE Url","AUSCERT Description","AUSCERT Url","BEA Description",
               "BEA Url","CALDERA Description","CALDERA Url","CERT Description","CERT Url",
               "CIAC Description","CIAC Url","CONECTIVA Description",
               "CONECTIVA Url","CONFIRM Description","CONFIRM Url","OSVDB Description",
               "OSVDB Url","BUGTRAQ Description", "BUGTRAQ Url","CISCO Description", 
               "CISCO Url","BINDVIEW Description", "BINDVIEW Url","EXPLOIT Description",
               "EXPLOIT Url","FEDORA Description", "FEDORA Url","FULLDISC Description",
               "FULLDISC Url","MILLWORM Description", "MILLWORM Url","MISC Description",
               "MISC Url","MLIST Description", "MLIST Url","SUSE Description",
               "SUSE Url","XF Description", "XF Url","UBUNTU Description", "UBUNTU Url",
              "VUPEN Description", "VUPEN Url","SREASON Description", "SREASON Url",
              "OVAL Description", "OVAL Url","SGI Description", "SGI Url","CHECKPOINT Description",
              "CHECKPOINT Url","MANDRAKE Description", "MANDRAKE Url","MANDRIVA Description", 
               "MANDRIVA Url","COMPAQ Description", "COMPAQ Url","FREEBSD Description", "FREEBSD Url",
              "HP Description", "HP Url","IBM Description", "IBM Url","IDEFENSE Description", 
               "IDEFENSE Url","IMMUNIX Description", "IMMUNIX Url","ISS Description", "ISS Url",
              "JVN Description", "JVN Url","L0PHT Description","L0PHT Url","OPENBSD Description",
               "OPENBSD Url","SUNALERT Description","SUNALERT Url","TRUSTIX Description",
               "TRUSTIX Url","SLACKWARE Description","SLACKWARE Url","NETBSD Description","NETBSD Url",
              "VIM Description","VIM Url","VULNWATCH Description","VULNWATCH Url","CERT_VN Description","CERT_VN Url",
               "MSKB Description","MSKB Url","NAI Description","NAI Url","SUN Description","SUN Url"]
header_file2= ["CVE ID","Reference Description","Reference Url"]

#creating list to hold data for file.write into file
file1_data = []
file2_data = []
count_reference ={}

On evaluation, the data was found to have plenty of Unicode errors. This module performs the read write operation into files are filteration. The unicode errors can be either printed or found in a file called "UnicodeErrors.txt"

In [233]:
#write into file currently holding references
def write_file(filename,data,header):
            with open(filename , 'w') as file:
                writer = csv.DictWriter(file, fieldnames = header)
                writer.writeheader()
                for value in data:
                    #print(value);
                    try:
                        writer.writerow(value)
                    except UnicodeEncodeError:
                        writer.writerow({k:v.encode('utf8') for k,v in value.items()})

File1 holds all accurately filtered entries and File2 contain the other entries. The function reference_sort performs filtering to decide what entry goes to what file. Python does not allow for switch cases, so we use if and elif. Here we have performed regex matching to seperate out the different types.

In [234]:
def counter(reference):
    if not reference in count_reference:
        count_reference[reference]=1
    else:
        count_reference[reference]+=1

In [235]:
def reference_sort(data):
    #data[1] holds references
    if data[1] is not None:
        for child in data[1].findall(table_root+"Reference"):
            file1 = {}
            file2 = {}
            #re.search(regex,text)            
            if re.search(BID_regex,child.find(table_root + "Description").text):
                counter("BID")
                file1["CVE ID"] = data[0].text
                file1["BID Url"] = child.find(table_root + "URL").text
                file1["BID Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(SECTRACK_regex,child.find(table_root + "Description").text):
                counter("SECTRACK")
                file1["CVE ID"] = data[0].text
                file1["SECTRACK Url"] = child.find(table_root + "URL").text
                file1["SECTRACK Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(MS_regex,child.find(table_root + "Description").text):
                counter("MS")
                file1["CVE ID"] = data[0].text
                file1["MS Url"] = child.find(table_root + "URL").text
                file1["MS Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "DEBIAN:" in child.find(table_root + "Description").text:
                counter("DEBIAN")
                file1["CVE ID"] = data[0].text
                file1["DEBIAN Url"] = child.find(table_root + "URL").text
                file1["DEBIAN Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)            
            elif re.search(REDHAT_regex,child.find(table_root + "Description").text):
                counter("REDHAT")
                file1["CVE ID"] = data[0].text
                file1["REDHAT Url"] = child.find(table_root + "URL").text
                file1["REDHAT Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(AIXAPAR_regex,child.find(table_root + "Description").text):
                counter("AIXAPAR")
                file1["CVE ID"] = data[0].text
                file1["AIXAPAR Url"] = child.find(table_root + "URL").text
                file1["AIXAPAR Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "GENTOO:" in child.find(table_root + "Description").text:
                counter("GENTOO")
                file1["CVE ID"] = data[0].text
                file1["GENTOO Url"] = child.find(table_root + "URL").text
                file1["GENTOO Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(SECUNIA_regex,child.find(table_root + "Description").text):
                counter("SECUNIA")
                file1["CVE ID"] = data[0].text
                file1["SECUNIA Url"] = child.find(table_root + "URL").text
                file1["SECUNIA Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "TURBO:" in child.find(table_root + "Description").text:
                counter("TURBO")
                file1["CVE ID"] = data[0].text
                file1["TURBO Url"] = child.find(table_root + "URL").text
                file1["TURBO Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(ALLAIRE_regex,child.find(table_root + "Description").text):
                counter("ALLAIRE")
                file1["CVE ID"] = data[0].text
                file1["ALLAIRE Url"] = child.find(table_root + "URL").text
                file1["ALLAIRE Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(APPLE_regex,child.find(table_root + "Description").text):
                counter("APPLE")
                file1["CVE ID"] = data[0].text
                file1["APPLE Url"] = child.find(table_root + "URL").text
                file1["APPLE Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "ATSTAKE:" in child.find(table_root + "Description").text:
                counter("ATSTAKE")
                file1["CVE ID"] = data[0].text
                file1["ATSTAKE Url"] = child.find(table_root + "URL").text
                file1["ATSTAKE Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(AUSCERT_regex,child.find(table_root + "Description").text):
                counter("AUSCERT")
                file1["CVE ID"] = data[0].text
                file1["AUSCERT Url"] = child.find(table_root + "URL").text
                file1["AUSCERT Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(CALDERA_regex,child.find(table_root + "Description").text):
                counter("CALDERA")
                file1["CVE ID"] = data[0].text
                file1["CALDERA Url"] = child.find(table_root + "URL").text
                file1["CALDERA Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(BEA_regex,child.find(table_root + "Description").text):
                counter("BEA")
                file1["CVE ID"] = data[0].text
                file1["BEA Url"] = child.find(table_root + "URL").text
                file1["BEA Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)            
            elif re.search(CIAC_regex,child.find(table_root + "Description").text):
                counter("CIAC")
                file1["CVE ID"] = data[0].text
                file1["CIAC Url"] = child.find(table_root + "URL").text
                file1["CIAC Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(CONECTIVA_regex,child.find(table_root + "Description").text):
                counter("CONECTIVA")
                file1["CVE ID"] = data[0].text
                file1["CONECTIVA Url"] = child.find(table_root + "URL").text
                file1["CONECTIVA Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(CERT_VN_regex,child.find(table_root + "Description").text):
                counter("CERT-VN")
                file1["CVE ID"] = data[0].text
                file1["CERT_VN Url"] = child.find(table_root + "URL").text
                file1["CERT_VN Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(CERT_regex,child.find(table_root + "Description").text):
                counter("CERT")
                file1["CVE ID"] = data[0].text
                file1["CERT Url"] = child.find(table_root + "URL").text
                file1["CERT Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "BINDVIEW:" in child.find(table_root + "Description").text:
                counter("BINDVIEW")
                file1["CVE ID"] = data[0].text
                file1["BINDVIEW Url"] = child.find(table_root + "URL").text
                file1["BINDVIEW Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "CHECKPOINT:" in child.find(table_root + "Description").text:
                counter("CHECKPOINT")
                file1["CVE ID"] = data[0].text
                file1["CHECKPOINT Url"] = child.find(table_root + "URL").text
                file1["CHECKPOINT Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)            
            elif re.search(CONFIRM_regex,child.find(table_root + "Description").text):
                counter("CONFIRM")
                file1["CVE ID"] = data[0].text
                file1["CONFIRM Url"] = child.find(table_root + "URL").text
                file1["CONFIRM Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(OSVDB_regex,child.find(table_root + "Description").text):
                counter("OSVDB")
                file1["CVE ID"] = data[0].text
                file1["OSVDB Url"] = child.find(table_root + "URL").text
                file1["OSVDB Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(BUGTRAQ_regex,child.find(table_root + "Description").text):
                counter("BUGTRAQ")
                file1["CVE ID"] = data[0].text
                file1["BUGTRAQ Url"] = child.find(table_root + "URL").text
                file1["BUGTRAQ Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(CISCO_regex,child.find(table_root + "Description").text):
                counter("CISCO")
                file1["CVE ID"] = data[0].text
                file1["CISCO Url"] = child.find(table_root + "URL").text
                file1["CISCO Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(EXPLOIT_regex,child.find(table_root + "Description").text):
                counter("EXPLOIT")
                file1["CVE ID"] = data[0].text
                file1["EXPLOIT Url"] = child.find(table_root + "URL").text
                file1["EXPLOIT Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(FEDORA_regex,child.find(table_root + "Description").text):
                counter("FEDORA")
                file1["CVE ID"] = data[0].text
                file1["FEDORA Url"] = child.find(table_root + "URL").text
                file1["FEDORA Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(FULLDISC_regex,child.find(table_root + "Description").text):
                counter("FULLDISC")
                file1["CVE ID"] = data[0].text
                file1["FULLDISC Url"] = child.find(table_root + "URL").text
                file1["FULLDISC Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "MILW0RM:" in child.find(table_root + "Description").text:
                counter("MILLWORM")
                file1["CVE ID"] = data[0].text
                file1["MILLWORM Url"] = child.find(table_root + "URL").text
                file1["MILLWORM Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(MISC_regex,child.find(table_root + "Description").text):
                counter("MISC")
                file1["CVE ID"] = data[0].text
                file1["MISC Url"] = child.find(table_root + "URL").text
                file1["MISC Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "MLIST:" in child.find(table_root + "Description").text:
                counter("MLIST")
                file1["CVE ID"] = data[0].text
                file1["MLIST Url"] = child.find(table_root + "URL").text
                file1["MLIST Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(OVAL_regex,child.find(table_root + "Description").text):
                counter("OVAL")
                file1["CVE ID"] = data[0].text
                file1["OVAL Url"] = child.find(table_root + "URL").text
                file1["OVAL Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "SGI:" in child.find(table_root + "Description").text:
                counter("SGI")
                file1["CVE ID"] = data[0].text
                file1["SGI Url"] = child.find(table_root + "URL").text
                file1["SGI Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "SREASON:" in child.find(table_root + "Description").text:
                counter("SREASON")
                file1["CVE ID"] = data[0].text
                file1["SREASON Url"] = child.find(table_root + "URL").text
                file1["SREASON Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "SUSE:" in child.find(table_root + "Description").text:
                counter("SUSE")
                file1["CVE ID"] = data[0].text
                file1["SUSE Url"] = child.find(table_root + "URL").text
                file1["SUSE Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "UBUNTU:" in child.find(table_root + "Description").text:
                counter("UBUNTU")
                file1["CVE ID"] = data[0].text
                file1["UBUNTU Url"] = child.find(table_root + "URL").text
                file1["UBUNTU Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "VUPEN:" in child.find(table_root + "Description").text:
                counter("VUPEN")
                file1["CVE ID"] = data[0].text
                file1["VUPEN Url"] = child.find(table_root + "URL").text
                file1["VUPEN Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "XF:" in child.find(table_root + "Description").text:
                counter("XF")
                file1["CVE ID"] = data[0].text
                file1["XF Url"] = child.find(table_root + "URL").text
                file1["XF Description"] = child.find(table_root + "Description").text
                file1_data.append(file1) 
            elif "COMPAQ" in child.find(table_root + "Description").text:
                counter("COMPAQ")
                file1["CVE ID"] = data[0].text
                file1["COMPAQ Url"] = child.find(table_root + "URL").text
                file1["COMPAQ Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "FREEBSD" in child.find(table_root + "Description").text:
                counter("FREEBSD")
                file1["CVE ID"] = data[0].text
                file1["FREEBSD Url"] = child.find(table_root + "URL").text
                file1["FREEBSD Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "HP:" in child.find(table_root + "Description").text:
                counter("HP")
                file1["CVE ID"] = data[0].text
                file1["HP Url"] = child.find(table_root + "URL").text
                file1["HP Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(IBM_regex,child.find(table_root + "Description").text):
                counter("IBM")
                file1["CVE ID"] = data[0].text
                file1["IBM Url"] = child.find(table_root + "URL").text
                file1["IBM Description"] = child.find(table_root + "Description").text
                file1_data.append(file1) 
            elif "IDEFENSE" in child.find(table_root + "Description").text:
                counter("IDEFENSE")
                file1["CVE ID"] = data[0].text
                file1["IDEFENSE Url"] = child.find(table_root + "URL").text
                file1["IDEFENSE Description"] = child.find(table_root + "Description").text
                file1_data.append(file1) 
            elif "IMMUNIX" in child.find(table_root + "Description").text:
                counter("IMMUNIX")
                file1["CVE ID"] = data[0].text
                file1["IMMUNIX Url"] = child.find(table_root + "URL").text
                file1["IMMUNIX Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "ISS" in child.find(table_root + "Description").text:
                counter("ISS")
                file1["CVE ID"] = data[0].text
                file1["ISS Url"] = child.find(table_root + "URL").text
                file1["ISS Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "JVN" in child.find(table_root + "Description").text:
                counter("JVN")
                file1["CVE ID"] = data[0].text
                file1["JVN Url"] = child.find(table_root + "URL").text
                file1["JVN Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "L0PHT" in child.find(table_root + "Description").text:
                counter("L0PHT")
                file1["CVE ID"] = data[0].text
                file1["L0PHT Url"] = child.find(table_root + "URL").text
                file1["L0PHT Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "MANDRAKE" in child.find(table_root + "Description").text:
                counter("MANDRAKE")
                file1["CVE ID"] = data[0].text
                file1["MANDRAKE Url"] = child.find(table_root + "URL").text
                file1["MANDRAKE Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "MANDRIVA" in child.find(table_root + "Description").text:
                counter("MANDRIVA")
                file1["CVE ID"] = data[0].text
                file1["MANDRIVA Url"] = child.find(table_root + "URL").text
                file1["MANDRIVA Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(OPENBSD_regex,child.find(table_root + "Description").text):
                counter("OPENBSD")
                file1["CVE ID"] = data[0].text
                file1["OPENBSD Url"] = child.find(table_root + "URL").text
                file1["OPENBSD Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "SUNALERT" in child.find(table_root + "Description").text:
                counter("SUNALERT")
                file1["CVE ID"] = data[0].text
                file1["SUNALERT Url"] = child.find(table_root + "URL").text
                file1["SUNALERT Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "SLACKWARE" in child.find(table_root + "Description").text:
                counter("SLACKWARE")
                file1["CVE ID"] = data[0].text
                file1["SLACKWARE Url"] = child.find(table_root + "URL").text
                file1["SLACKWARE Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "VULNWATCH" in child.find(table_root + "Description").text:
                counter("VULNWATCH")
                file1["CVE ID"] = data[0].text
                file1["VULNWATCH Url"] = child.find(table_root + "URL").text
                file1["VULNWATCH Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(VIM_regex,child.find(table_root + "Description").text):
                counter("VIM")
                file1["CVE ID"] = data[0].text
                file1["VIM Url"] = child.find(table_root + "URL").text
                file1["VIM Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "NETBSD" in child.find(table_root + "Description").text:
                counter("NETBSD")
                file1["CVE ID"] = data[0].text
                file1["NETBSD Url"] = child.find(table_root + "URL").text
                file1["NETBSD Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "MSKB" in child.find(table_root + "Description").text:
                counter("MSKB")
                file1["CVE ID"] = data[0].text
                file1["MSKB Url"] = child.find(table_root + "URL").text
                file1["MSKB Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif "TRUSTIX" in child.find(table_root + "Description").text:
                counter("TRUSTIX")
                file1["CVE ID"] = data[0].text
                file1["TRUSTIX Url"] = child.find(table_root + "URL").text
                file1["TRUSTIX Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)  
            elif re.search(NAI_regex,child.find(table_root + "Description").text):
                counter("NAI")
                file1["CVE ID"] = data[0].text
                file1["NAI Url"] = child.find(table_root + "URL").text
                file1["NAI Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            elif re.search(SUN_regex,child.find(table_root + "Description").text):
                counter("SUN")
                file1["CVE ID"] = data[0].text
                file1["SUN Url"] = child.find(table_root + "URL").text
                file1["SUN Description"] = child.find(table_root + "Description").text
                file1_data.append(file1)
            else:
                counter("OTHERS")
                file2["CVE ID"] = data[0].text
                file2["Reference Description"] = child.find(table_root + "Description").text
                file2["Reference Url"] = child.find(table_root + "URL").text
                file2_data.append(file2)

In [236]:
#verify write operation into file and perform reference sort
def module_runner(cve_Tree):
    print("Vulnerability data count:" + str(len(CVE_tree.findall(table_root+"Vulnerability")))); 
    v_counter = 0
    for vul in CVE_tree.findall(table_root+"Vulnerability"):
        #print ("Vulnerability index: " + str(v_counter));
        v_counter +=1
        reference_sort((vul.find(table_root + "CVE"),vul.find(table_root + "References")))

The module below performs parsing of all files under a particular folder called CVE_XML. This allows for flexibility to choose what files we want to filter(either particular years or cumulatively). This module also records values and counts of vulnerability entries, filtered and unfiltered data.

In [237]:
#call module runner to perform parsing
for filename in glob.glob("CVE_XML/*.xml"):
        CVE_tree = ElementTree()
        CVE_tree.parse(filename)
        module_runner(CVE_tree)
#print (count_reference);
print ("Filtered count: "+ str(len(file1_data)));
print ("Unfiltered count: "+ str(len(file2_data)));
percentage = ((len(file1_data)/(len(file1_data)+len(file2_data)))*100)
print ("percentage filtered:"+ str(percentage));
write_file('file1.csv', file1_data, header_file1)
write_file('file2.csv', file2_data, header_file2)


Vulnerability data count:103484
Filtered count: 497420
Unfiltered count: 1138
percentage filtered:99.77174170307165


In [241]:
#creating a histogram for cwe ID types by creating a dictionary
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
data = {}
data['Entries'] = count_reference
#saving in dictionary for sorting and visualising
df_data = pd.DataFrame(data).sort_values(by='Entries', ascending=True)
series = df_data.loc[:,'Entries']

p = figure(width=800, y_range=series.index.tolist(), title="Histogram of coverage of reference types")

p.xaxis.axis_label = 'Frequency'
p.xaxis.axis_label_text_font_size = '10pt'
p.xaxis.major_label_text_font_size = '6pt'

p.yaxis.axis_label = 'Reference name'
p.yaxis.axis_label_text_font_size = '10pt'
p.yaxis.major_label_text_font_size = '8pt'

j = 1
for k,v in series.iteritems():
  
  #Print fields, values, orders
  #print (k,v,j) 
  p.rect(x=v/2, y=j, width=abs(v), height=0.5,
    width_units="data", height_units="data")
  j += 1
show(p)

Loading BokehJS ...